# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [103]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [104]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [105]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_KEY")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [106]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
toronto_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)
data = df_neighbourhood_locations

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [107]:
# Getting the data from the top 10 expensive neighbourhoods
# YOUR CODE HERE!
dwelling_types = [
    "single_detached_house",
    "apartment_five_storeys_plus",
    "movable_dwelling",
    "semi_detached_house",
    "row_house",
    "duplex",
    "apartment_five_storeys_less",
    "other_house" 
]

df_expensive_neighbourhoods = toronto_data.groupby(by="neighbourhood").mean()
df_expensive_neighbourhoods = df_expensive_neighbourhoods.sort_values(
    by="average_house_value", ascending=False
).head(10)
df_expensive_neighbourhoods = df_expensive_neighbourhoods.reset_index()

# Calculate the mean number of dwelling types units per year
# YOUR CODE HERE!
df_dwelling_units = toronto_data[dwelling_types].groupby(toronto_data.index).sum()

# Calculate the average monthly shelter costs for owned and rented dwellings
# YOUR CODE HERE!
df_average_shelter_costs =(
    toronto_data[["shelter_costs_owned", "shelter_costs_rented"]]
    .groupby([toronto_data.index]).mean()
)
df_average_shelter_costs

,shelter_costs_owned,shelter_costs_rented
year,,
2001,846.878571,1085.935714
2006,1316.800000,925.414286
2011,1448.214286,1019.792857
2016,1761.314286,1256.321429


### Panel Visualization Functions

In [108]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    df_all_neighbourhoods = toronto_data.groupby(by= "neighbourhood").mean()
    df_all_neighbourhoods = df_all_neighbourhoods.reset_index()

    df_all_neighbourhoods = pd.merge(df_neighbourhood_locations, df_all_neighbourhoods, on="neighbourhood")
    
    
    map = px.scatter_mapbox(
        df_all_neighbourhoods,
        lat="lat",
        lon="lon",
        size='average_house_value',
        color='average_house_value',
        color_continuous_scale= px.colors.cyclical.IceFire,
        size_max=25,
        zoom=10,
        hover_name="neighbourhood",
        hover_data = [
            "shelter_costs_owned",
            "shelter_costs_rented",
            "single_detached_house",
            "apartment_five_storeys_plus",
            "movable_dwelling",
            "semi_detached_house",
            "row_house",
            "duplex",
            "apartment_five_storeys_less",
            "other_house"     
        ],
        title="Average House Value in Toronto",
        height=500
    )
    plotlypanel=pn.pane.Plotly(map)
    plotlypanel._updates=True
    return plotlypanel

neigbourhood_map_pane = pn.Column(neighbourhood_map())

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    
    # YOUR CODE HERE!
    
    fig = data.hvplot.bar(title=title).opts(xlabel=xlabel).opts(ylabel=ylabel).opts(color=color).opts(xrotation=90).opts(height=600)
    
        
    return fig
    
bar_chart = pn.Column(create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Types Units", "red"), 
    create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Types Units", "blue"),
    create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Types Units", "maroon"),
    create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Types Units", "purple"))


def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    # YOUR CODE HERE!
    fig = data.hvplot.line(title=title).opts(xlabel=xlabel).opts(ylabel=ylabel).opts(color=color)
    
    return fig
line_chart = pn.Row(create_line_chart(df_average_shelter_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto",'Year', "Avg Monthly Shelter Costs","red"),
    create_line_chart(df_average_shelter_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto",'Year', "Avg Monthly Shelter Costs","black"))
    

def average_house_value():
    """Average house values per year."""
    
    # YOUR CODE HERE!
    df_average_house_value =(
        toronto_data[["average_house_value"]]
        .groupby([toronto_data.index]).mean())

    average_house_value = df_average_house_value
    
    average_house_value = average_house_value.hvplot.line(
        "year",
        "average_house_value",
        xlabel = "Year",
        ylabel = "Avg House Value"
)
    return average_house_value
average_house_value = pn.Column(average_house_value)

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    # YOUR CODE HERE!
    df_average_value = toronto_data[["neighbourhood", "average_house_value"]]
    df_average_value = df_average_value.reset_index()
    average_value_by_neighbourhood = df_average_value
    
    average_value_by_neighbourhood = average_value_by_neighbourhood.hvplot.line(
        "year",
        "average_house_value",
        xlabel = "Year",
        ylabel = "Avg House Value",
        groupby = "neighbourhood",
)
    return average_value_by_neighbourhood
    
average_value_by_neighbourhood = pn.Column(average_value_by_neighbourhood)

def number_dwelling_types():
    """Number of dwelling types per year"""
    
    # YOUR CODE HERE!
    df_all_dwellings = toronto_data.reset_index()
    number_dwelling_types = df_all_dwellings
    
    number_dwelling_types = number_dwelling_types.hvplot.bar(
        "year",
        dwelling_types,
        xlabel = "Year",
        ylabel = "Dwelling Type Units",
        rot = 45,
        height = 600, 
        groupby = "neighbourhood" 
) 
    return number_dwelling_types

number_dwelling_types = pn.Column(number_dwelling_types)

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""

    # YOUR CODE HERE!
    average_house_value_snapshot = px.bar(
        toronto_data,
        x="neighbourhood",
        y="average_house_value",
        color= "average_house_value",
        height=1024,
        facet_row=toronto_data.index,
        title="Average House Values in Toronto per Neighbourhood"
             
)
    return average_house_value_snapshot

average_house_value_snapshot = pn.Column(average_house_value_snapshot)


def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    # YOUR CODE HERE!
    df_expensive_neighbourhoods = toronto_data.groupby(by="neighbourhood").mean()
    df_expensive_neighbourhoods = df_expensive_neighbourhoods.sort_values(
        by="average_house_value", ascending=False).head(10)
    df_expensive_neighbourhoods = df_expensive_neighbourhoods.reset_index()
    top_most_expensive_neighbourhoods = df_expensive_neighbourhoods


    top_most_expensive_neighbourhoods = top_most_expensive_neighbourhoods.hvplot.bar(
        "neighbourhood",
        "average_house_value",
        xlabel = "Neighbourhood",
        ylabel = "Avg House Value",
        title = "Top 10 Most Expensive Neighbourhoods",
        rot = 90,
        height = 600, 
        ylim = (0, 2000000)
)
    return top_most_expensive_neighbourhoods

expensive_neighbourhoods = pn.Column(top_most_expensive_neighbourhoods, "Top 10 Most Expensive Neighbourhoods", "Neighbourhood", "Avg House Value") 


def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!
    df_expensive_neighbourhoods_per_year = toronto_data[toronto_data["neighbourhood"].isin(df_expensive_neighbourhoods["neighbourhood"])]
    df_expensive_neighbourhoods_per_year = df_expensive_neighbourhoods_per_year.reset_index()
    expensive_neighbourhoods_per_year = df_expensive_neighbourhoods_per_year

    sunburts_cost_analysis = px.sunburst(
    expensive_neighbourhoods_per_year,
    path=["year", "neighbourhood"],
    values="average_house_value",
    color="shelter_costs_owned",
    hover_data=["shelter_costs_owned", "shelter_costs_rented"],
    color_continuous_scale="Greens",
    title="Costs Analysis of Most Expensive Neighbourhoods in Toronto per Year"
)
    return sunburts_cost_analysis

sunburts_cost_analysis = pn.Column(sunburts_cost_analysis, "Costs Analysis of Most Expensive Neighbourhoods in Toronto per Year")

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [109]:
# Create a Title for the Dashboard
# YOUR CODE HERE!

title = pn.pane.Markdown(
'''
Toronto's Real Estate Analysis from 2001-2016
''',
    width=800,)
# Define a welcome text
# YOUR CODE HERE!
welcome = pn.pane.Markdown(
'''
The dashbord shows the analysis of the historical house value, dwelling types and cost in Toronto between 2001 and 2016 in a visual form. The tabs will help navigate 
through the dashbord to explore the information across the stated years
''',
    width=1024,
)

# Create a tab layout for the dashboard
# YOUR CODE HERE!
Toronto_tab = pn.Tabs(
    ("Welcome", pn.Column(welcome ,neigbourhood_map_pane)),
    ("Yearly Market Analysis", bar_chart),
    ("Shelter Costs vs House Value", pn.Column(line_chart, average_house_value)),
    ("Neighbourhood Analysis", pn.Row(average_value_by_neighbourhood, number_dwelling_types, average_house_value_snapshot)),
    ("Top Expensive Neighbourhood", pn.Row(expensive_neighbourhoods, sunburts_cost_analysis))
    
)

# Create the main dashboard
# YOUR CODE HERE!
dashboard = pn.Column(pn.Row(title), Toronto_tab)

## Serve the Panel Dashboard

In [110]:
dashboard.servable()

Column
    [0] Row
        [0] Markdown(str, width=800)
    [1] Tabs
        [0] Column
            [0] Markdown(str, width=1024)
            [1] Column
                [0] Plotly(Figure)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
            [2] HoloViews(Bars)
            [3] HoloViews(Bars)
        [2] Column
            [0] Row
                [0] HoloViews(Curve)
                [1] HoloViews(Curve)
            [1] Column
                [0] Column
                    [0] Column()
                    [1] Row
                        [0] HoloViews(Curve, name='interactive38655')
        [3] Row
            [0] Column
                [0] Column
                    [0] Column()
                    [1] Row
                        [0] Row(name='interactive38738')
                            [0] HoloViews(DynamicMap, name='interactive38738')
                            [1] Column
                                [0] WidgetBox
                                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                                [1] VSpacer()
            [1] Column
                [0] Column
                    [0] Column()
                    [1] Row
                        [0] Row(name='interactive38774')
                            [0] HoloViews(DynamicMap, name='interactive38774')
                            [1] Column
                                [0] WidgetBox
                                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                                [1] VSpacer()
            [2] Column
                [0] Column
                    [0] Column()
                    [1] Row
                        [0] Plotly(Figure, name='interactive38820')
        [4] Row
            [0] Column
                [0] Column
                    [0] Column()
                    [1] Row
                        [0] HoloViews(Bars, name='interactive38827')
                [1] Markdown(str)
                [2] Markdown(str)
                [3] Markdown(str)
            [1] Column
                [0] Column
                    [0] Column()
                    [1] Row
                        [0] Plotly(Figure, name='interactive38927')
                [1] Markdown(str)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [111]:
neighbourhood_map()

Plotly(Figure)

In [112]:
bar_chart

# # Bar chart for 2001
#create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
#create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
#create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "maroon")

# # Bar chart for 2016
#create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "purple")

Column
    [0] HoloViews(Bars)
    [1] HoloViews(Bars)
    [2] HoloViews(Bars)
    [3] HoloViews(Bars)

In [113]:
line_chart

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

Row
    [0] HoloViews(Curve)
    [1] HoloViews(Curve)

In [114]:
average_house_value

Column
    [0] Column
        [0] Column()
        [1] Row
            [0] HoloViews(Curve, name='interactive38655')

In [115]:
average_value_by_neighbourhood

Column
    [0] Column
        [0] Column()
        [1] Row
            [0] Row(name='interactive38738')
                [0] HoloViews(DynamicMap, name='interactive38738')
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                    [1] VSpacer()

In [116]:
number_dwelling_types

Column
    [0] Column
        [0] Column()
        [1] Row
            [0] Row(name='interactive38774')
                [0] HoloViews(DynamicMap, name='interactive38774')
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                    [1] VSpacer()

In [117]:
average_house_value_snapshot

Column
    [0] Column
        [0] Column()
        [1] Row
            [0] Plotly(Figure, name='interactive38820', relayout_data={'autosize': True}, viewport={'xaxis.range': [-0.5, ...})

In [118]:
expensive_neighbourhoods

Column
    [0] Column
        [0] Column()
        [1] Row
            [0] HoloViews(Bars, name='interactive38827')
    [1] Markdown(str)
    [2] Markdown(str)
    [3] Markdown(str)

In [119]:
sunburts_cost_analysis

Column
    [0] Column
        [0] Column()
        [1] Row
            [0] Plotly(Figure, name='interactive38927', relayout_data={'autosize': True})
    [1] Markdown(str)